# Strategy based on RSI indicator.¶

1. Buy the instrument such as Nifty or SPY when the RSI is less than 30
2. Exit conditions: a. Take profit of 5% or RSI > 70, b. Stop loss of - 2%
3. Optional: Optimize the strategy by adjusting the RSI value. Also, take profit and stop loss criteria can be different for each stock.
4. Note: You can use TA-Lib in Python to compute the RSI value.

Import the required libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import pandas_datareader as pdr
import talib
%matplotlib inline
import warnings

warnings.filterwarnings('ignore')

In [5]:
#Define the backtesting period
# start date 
start = datetime.datetime(2017,1,25)
#end date
end = datetime.datetime(2021,1,26)

In [16]:
#Create a backtesting class 
class backtest_RSI:
    
    # Create instance attributes/methods
    def __init__(self,ticker,start,end,TP_level,SL_level):
        
        #Create Instance attributes
        self.ticker = ticker
        self.start = start
        self.end = end
        self.TP_level = TP_level
        self.SL_level = SL_level
    
        #Create Instance Methods
        self.fetch_data()
        self.indicators()
        self.process()
        self.signals()
        self.positions()
        self.returns()
    
    # Fetch the data from yahoo finance
    def fetch_data(self):
        self.df = pdr.get_data_yahoo(self.ticker,self.start,self.end)
    
    # Compute the RSI indicator of range 14 days
    def indicators(self):
        
        self.df['RSI'] = talib.RSI(self.df['Adj Close'],timeperiod = 14)
    
    #Clean the data by dropping the missing values 
    def process(self):
        self.df.drop(['High', 'Low', 'Open', 'Close', 'Volume'],axis=1,inplace=True)
        self.df.dropna(inplace=True,axis=0)
        self.df = self.df.reset_index()
   
    # To get the entry and exit signal
    def signals(self):
        
        self.df['entry_price'] = 0
        self.df['RSI_signal'] = 0
        
        n = self.df.shape[0]
        a=1
        b=0
        
        #This loop computes the signal for both entry and exit based on RSI indicator
        for i in range(0,n):
            
            if self.df['RSI'].loc[i] <= 30 and a == 1:
                self.df['RSI_signal'].loc[i] = 1
                self.df['entry_price'].loc[i] = self.df['Adj Close'].loc[i]
                
                a=0
                b=1
                
            if b==1 and self.df['RSI'].loc[i] >=70:
                self.df['RSI_signal'].loc[i] = -1
                
                a=1
                b=0
                
        self.df['take_profit_price'] = self.df['entry_price']*(1+ (self.TP_level/100))
        self.df['stop_loss_price'] = self.df['entry_price']*(1- (self.SL_level/100))
        
        self.df['take_profit_price']=self.df['take_profit_price'].replace(to_replace=0,method='ffill')
        self.df['stop_loss_price']=self.df['stop_loss_price'].replace(to_replace=0,method='ffill')
        
        #This loop updates the exit signal considering the stoploss additionally
        for i in range(0,n):
            if self.df['Adj Close'].loc[i] < self.df['stop_loss_price'].loc[i] or self.df['Adj Close'].loc[i] > self.df['take_profit_price'].loc[i]:
                self.df['RSI_signal'].loc[i] = -1
                
        self.df['RSI_signal']=self.df['RSI_signal'].replace(to_replace=0,method='ffill')
    
    # Mark the positions taken
    def positions(self):
        
        self.df['positions']= self.df['RSI_signal'].replace(to_replace=-1,value=0)
        self.df['positions'] = self.df['positions'].shift()
    
    # Generate the returns
    def returns(self):
        
        #BnH returns are the returns generated by simply buying and holding the instrument
        self.df['BnH_Returns'] = np.log(self.df['Adj Close']/self.df['Adj Close'].shift())
        #Strategy Returns are the returns generated by implementing this strategy
        self.df['Strategy_Returns'] = self.df['BnH_Returns']*self.df['positions']
        BnH_Returns = self.df['BnH_Returns'].sum()
        Strat_Returns = self.df['Strategy_Returns'].sum()
        print('Total Strategy Returns:', self.df['Strategy_Returns'].sum())
        print('Total Buy and Hold Returns:', self.df['BnH_Returns'].sum())
        return Strat_Returns, BnH_Returns        
    
                
                
        

In [18]:
indice_list=['^NSEI','SPY','^CNXIT','^NSEBANK','^HSI','^GSPC']


indices=[]
Returns =[]

for i in indice_list:
    a = backtest_RSI(i,start,end,5,2)
    indices.append(i)
    Returns.append(a.returns())
    
Strat_Returns = [x[0] for x in Returns]
BnH_Returns = [x[1] for x in Returns]

Total Strategy Returns: -0.12599617322811085
Total Buy and Hold Returns: 0.4898195773134017
Total Strategy Returns: -0.12599617322811085
Total Buy and Hold Returns: 0.4898195773134017
Total Strategy Returns: -0.04714745982740475
Total Buy and Hold Returns: 0.575377466937519
Total Strategy Returns: -0.04714745982740475
Total Buy and Hold Returns: 0.575377466937519
Total Strategy Returns: 0.08291033313756646
Total Buy and Hold Returns: 0.9145313510971267
Total Strategy Returns: 0.08291033313756646
Total Buy and Hold Returns: 0.9145313510971267
Total Strategy Returns: 0.009794956731980758
Total Buy and Hold Returns: 0.4364829046059203
Total Strategy Returns: 0.009794956731980758
Total Buy and Hold Returns: 0.4364829046059203
Total Strategy Returns: 0.05153345487840502
Total Buy and Hold Returns: 0.19816608412790768
Total Strategy Returns: 0.05153345487840502
Total Buy and Hold Returns: 0.19816608412790768
Total Strategy Returns: 0.00827020956906576
Total Buy and Hold Returns: 0.5028575601

In [19]:
Results = pd.DataFrame({'Indice':indices,'Returns':Strat_Returns})
Results.sort_values(by='Returns',ascending=False)

,Indice,Returns
2,^CNXIT,0.082910
4,^HSI,0.051533
3,^NSEBANK,0.009795
5,^GSPC,0.008270
1,SPY,-0.047147
0,^NSEI,-0.125996
